## Install detectron2

In [ ]:
!pip install pyyaml==5.1
!pip install torch==1.10.0+cu111 torchvision==0.11.1+cu111 torchaudio===0.10.0+cu111 -f https://download.pytorch.org/whl/cu111/torch_stable.html 
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.10/index.html

import torch

TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
# Install detectron2 that matches the above pytorch version
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
#!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/$CUDA_VERSION/torch$TORCH_VERSION/index.html
# If there is not yet a detectron2 release that matches the given torch + CUDA version, you need to install a different pytorch.

# exit(0)  # After installation, you may need to "restart runtime" in Colab. This line can also restart runtime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/cu111/torch_stable.html
     |████████████▌                   | 834.1 MB 96.9 MB/s eta 0:00:14tcmalloc: large alloc 1147494400 bytes == 0x38e94000 @  0x7fb8dfa9d615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |███████████████▉                | 1055.7 MB 71.7 MB/s eta 0:00:16tcmalloc: large alloc 1434370048 bytes == 0x7d4ea000 @  0x7fb8dfa9d615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.10/index.html
     |████████████████████████████████| 7.0 MB 578 kB/s 
     |████████████████████████████████| 79 kB 3.8 MB/s 
     |████████████████████████████████| 50 kB 6.8 MB/s 
     |████████████████████████████████| 130 kB 10.7 MB/s 
     |████████████████████████████████| 151 kB 68.3 MB/s 
     |████████████████████████████████| 843 kB 57.9 MB/s 
     |████████████████████████████████| 117 kB 71.7 MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.5.post20220512-py3-none-any.whl size=61288 sha256=a5cb809c16598b175f8de8990ca9753b77b0014c73bba64fed4c8cd4deea6408
  Stored in directory: /root/.cache/pip/wheels/68/20/f9/a11a0dd63f4c13678b2a5ec488e48078756505c7777b75b29e
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144575 sha256=3089a694401c4499c7

torch:  1.11 ; cuda:  cu113


## Import

In [ ]:
#import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor

# import some common libraries
import numpy as np
import os, json, cv2, random
import copy
from google.colab.patches import cv2_imshow
from google.colab import files
import shutil
import PIL
from PIL import Image
from matplotlib import pyplot as plt
from pathlib import Path
import imutils

## All the functions we need

In [ ]:
def custom_config(model_path = None, BATCH_SIZE_PER_IMAGE = 512, BASE_LR = 0.00025, MAX_ITER = 1000):

  cfg = get_cfg()

  # get configuration from model_zoo
  config_file = "COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"
  cfg.merge_from_file(model_zoo.get_config_file(config_file))

  # initialize weights
  if not model_path:
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(config_file) 
  else:
    assert os.path.isfile(model_path), '.pth file not found'
    cfg.MODEL.WEIGHTS = model_path

  cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (quadrat). 
  cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7  # set a custom testing threshold
  
  return cfg

In [ ]:
def find_contour(mask):
  # find the contours of the mask
  cnts = cv2.findContours(mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
  cnts = imutils.grab_contours(cnts)
  # keep only the biggest one
  cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:1]
  cnts = cnts[0]

  assert cnts.shape[1:] == (1,2), "cnts.shape[1:] should be (1,2)"

  return cnts

In [ ]:
def minimum_area_enclosing(cnts):
  '''
  Find the minimum-area rotated rectangle that encloses the given contour
  '''
  rect = cv2.minAreaRect(cnts)
  box = cv2.boxPoints(rect)
  box = np.int0(box)
  screenCnt = np.array([[i] for i in box.tolist()])
  #print('Estimated corners:', screenCnt.tolist())
  return screenCnt

In [ ]:
def line(p1, p2):
  '''
  Produces coefs A, B, C of line equation by two points provided
  Ex: L1 = line([2,3], [4,0]) --> L1 = (3, 2, 12)
  '''
  A = (p1[1] - p2[1])
  B = (p2[0] - p1[0])
  C = (p1[0]*p2[1] - p2[0]*p1[1])
  return A, B, -C

def intersection(L1, L2):
  '''
  Finds intersection point (if any) of two lines provided by coefs.
  Ex: L1 = line([0,1], [2,3])
      L2 = line([2,3], [4,0])
      intersec = intersection(L1,L2) --> intersec = [2, 3]

      L1 = line([0,1], [2,3])
      L2 = line([0,2], [2,4])
      intersect = intersection(L1,L2) --> None
  '''
  D  = L1[0] * L2[1] - L1[1] * L2[0]
  Dx = L1[2] * L2[1] - L1[1] * L2[2]
  Dy = L1[0] * L2[2] - L1[2] * L2[0]

  if D != 0:
    x = Dx / D
    y = Dy / D
    return [int(x), int(y)]
  else:
    return None

In [ ]:
def get_lines_intersections(pts, ncols = 1e158, nrows = 1e158):
  '''
  Input: A list of 4 consecutive lines
         Ex: [[[1,2], [3,4]], # line 1 passes through point [1,2] and [3,4]
              [[5,6], [7,8]],
              [[9,10], [11,12]],
              [[13,14], [15,16]]]

  Compute: The intersection between each consecutive lines
           If the intersection falls outside the image, it is projected on the image

  Return: A new contour with the 4 line intersections/the 4 new corners
          Ex: array([[[3, 0]],
                    [[3, 3]],
                    [[0, 3]],
                    [[0, 0]]], dtype=int32)

  '''
  C = [0, 1, 2, 3, 0] 
  newCnt = []

  assert len(pts) == 4, 'There are more than 4 points in the input list'

  for corner in range(4):

    # First line
    p1 = pts[C[corner]]
    L1 = line(p1[0], p1[1])

    # Second line 
    p2 = pts[C[corner+1]]
    L2 = line(p2[0], p2[1])
    
    # Find intersection
    inter = intersection(L1, L2)
    assert inter, 'could not find any intersection between 2 lines'

    # Projection on the image
    inter = [max(0,min(ncols, inter[0])), max(0,min(nrows, inter[1]))]

    newCnt.append([inter])

  newCnt = np.array(newCnt, dtype = "int32")
  #print('New estimated corners:', newCnt.tolist())

  return newCnt

In [ ]:
def fit_lines_on_edges(cnts, screenCnt, rows, cols, threshold = 100, epsilon = 100):
  '''
  Input:
  - cnts: the full contour 
  - screenCnt: 4 estimated corners of the contour, obtained with methods 1 or 2

  Steps:
  - For each side, delimited by 2 consecutive corners of screenCnt (c1 and c2)
    - Find all the points of cnts that are located between c1 and c2 (+- margin)
    - Fit a line, using cv2.fitLine, on these points 

  Output:
  - pts: the list of the 4 fitted lines
         Ex: [[[1,2], [3,4]], # line 1 passes through point [1,2] and [3,4]
              [[5,6], [7,8]],
              [[9,10], [11,12]],
              [[13,14], [15,16]]]
  '''

  assert screenCnt.shape == (4, 1, 2), 'screenCnt.shape should be (4, 1, 2)'

  X = [i[0][0] for i in cnts] 
  Y = [i[0][1] for i in cnts] 

  lefty = [] 
  righty = []
  leftx = []
  rightx = []
  cutCnt = []
  C = [0, 1, 2, 3, 0]

  for edge in range(4):
    # define 2 extreme points of the edge
    c1 = screenCnt[C[edge]][0]
    c2 = screenCnt[C[edge+1]][0]

    # select all the contour points that are located between these 2 points +- margin --> cutCnt
    if abs(c1[0]-c2[0]) < threshold: # if the 2 pts have almost the same abscissa
      c_mean = (c1[0] + c2[0])/2
      X_margin = [max(0, c_mean - epsilon), min(c_mean + epsilon, cols)]
      Y_margin = sorted([c1[1], c2[1]])

    elif abs(c1[1]-c2[1]) < threshold: # if the 2 pts have almost the same ordinate
      c_mean = (c1[1] + c2[1])/2
      X_margin = sorted([c1[0], c2[0]])
      Y_margin = [max(0,c_mean - epsilon), min(c_mean + epsilon, rows)] 

    else:
      X_margin = sorted([c1[0], c2[0]])
      Y_margin = sorted([c1[1], c2[1]])

    c = [[[X[ind], Y[ind]]] for ind in range(len(X)) if X_margin[0] <= X[ind] <= X_margin[1] and Y_margin[0] <= Y[ind] <= Y_margin[1]]
    cutCnt.append(np.array(c))

    # fit a line on c
    [vx, vy, x, y] = cv2.fitLine(np.array(c), cv2.DIST_L2, 0, 0.01, 0.01)

    # append the result for this edge to left and right points
    leftx.append(0)
    rightx.append(cols)
    lefty.append(int((-x*vy/vx) + y))
    righty.append(int(((cols-x)*vy/vx)+y))

  # 
  pts = [[[leftx[ind], lefty[ind]], [rightx[ind], righty[ind]]] for ind in range(4)]
  return pts

In [ ]:
def transform_perspective(img, pts, newSize = 500):
  '''
  example pts = [[1170,200], [3760,100], [3890,2700], [1250,2800]]
  coordinates are ordered such that the first entry in the list is the top-left,
	the second entry is the top-right, the third is the bottom-right, 
  and the fourth is the bottom-left
  '''

  # compute the perspective transform matrix
  dst = np.array([
		[0, 0],
    [0, newSize],
    [newSize, newSize],
		[newSize, 0]], dtype = "float32")
  
  M = cv2.getPerspectiveTransform(pts, dst)

  # apply the transformation matrix 
  warped = cv2.warpPerspective(img, M, (newSize, newSize))

  # return
  return warped

In [ ]:
def main(img_path, model_path):
  
  cfg = custom_config(model_path=model_path)
  predictor = DefaultPredictor(cfg)

  # read image
  img = cv2.imread(img_path)
  rows, cols = img.shape[:2]

  # prediction
  output = predictor(img) 

  # get mask
  mask_array = 1*output["instances"].get('pred_masks').to('cpu').numpy()
  mask_array = np.moveaxis(mask_array, 0, -1)
  mask_array = np.repeat(mask_array, 3, axis=2)
  mask3d = np.where(mask_array==False, 0, 
          (np.where(mask_array==True, 255, img)))
  
  #mask3d = Image.fromarray(mask3d)
  mask = mask3d[:,:,1]

  # Find contour of the mask
  cnts = find_contour(mask)

  ## Fist step : First corners estimations
  screenCnt = minimum_area_enclosing(cnts)

  ## Second step: Fit 4 lines on each edges
  lines_pts = fit_lines_on_edges(cnts, screenCnt, rows, cols)
  
  ## Third step: Lines intersection to get corners
  newCnt = get_lines_intersections(lines_pts, ncols = cols, nrows = rows)

  ## Last step: Reframing
  warped = transform_perspective(img, pts = np.float32(newCnt[:,0]), newSize = 400)

  return warped

## Demo

In [ ]:
# Mount the drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
def get_dataset_dicts(img_dir):
    dataset_dicts = []
    img_dir = Path(img_dir)
    for i, img_path in enumerate([*img_dir.glob("*.jpg")]):
        dataset_dicts.append(str(img_path))
    
    return dataset_dicts

In [ ]:
# Path of testing images
imgs = get_dataset_dicts("/content/drive/MyDrive/DSTI/coral_reef/img")

In [ ]:
# The path of the saved final model
model_path_1 = "/content/drive/MyDrive/DSTI/coral_reef/mlruns/0/e23af3d36ced4c0095619b32eee9e622/artifacts/model_final.pth"
model_path_2 = "/content/drive/MyDrive/DSTI/coral_reef/mlruns/0/7bd7d8b521ae4ca0a3c964a874f5d768/artifacts/model_final.pth"
model_path_3 = "/content/drive/MyDrive/DSTI/coral_reef/mlruns/0/02c3f409fa08455d9f58ee6364c86865/artifacts/model_final.pth" 

In [ ]:
# Reframing
N = len(imgs)
for img_path in random.sample(imgs, 3):
  print(img_path)
  img_reframed_1 = main(img_path=img_path, model_path=model_path_1)
  img_reframed_2 = main(img_path=img_path, model_path=model_path_2)
  img_reframed_3 = main(img_path=img_path, model_path=model_path_3)

  cv2_imshow(cv2.hconcat([img_reframed_1, img_reframed_2, img_reframed_3]))

#cv2_imshow(img_reframed)